In [ ]:
%aiida

In [ ]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt

import ase.io

from aiida.engine import submit, run_get_node, run
from aiida.orm import Int, Float, Str, Bool, Code, Dict, List
from aiida.orm import StructureData, RemoteData, FolderData

GaussianSpinWorkChain = WorkflowFactory('nanotech_empa.gaussian.spin')

import aiida_nanotech_empa.utils.gaussian_spin_wc_postprocess as pp

In [ ]:
ase_geom = ase.io.read("./hexacene.xyz")
ase_geom.cell = np.diag([10.0, 10.0, 10.0])

struct_node = StructureData(ase=ase_geom)

# submit

In [ ]:
wc_node = submit(
    GaussianSpinWorkChain,
    gaussian_code=load_code("gaussian09@euler"),
    formchk_code=load_code("formchk@euler"),
    cubegen_code=load_code("cubegen@euler"),
    structure=struct_node,
    functional=Str('B3LYP'),
    basis_set_opt=Str('6-31G*'),
    basis_set_scf=Str('6-31+G*'),
    multiplicity_list=List(list=[0, 1, 3]),
)

In [ ]:
wc_node.pk

# monitor

In [ ]:
def wc_overview(wc_node, indent=""):
    if wc_node.is_excepted:
        info = "**EXCEPTED**"
    elif not wc_node.is_terminated:
        info = "RUNNING"
    else:
        info = ""
    print(indent + "%d %s %s %s" % (wc_node.pk, wc_node.description, wc_node.process_label, info))
    for c in reversed(wc_node.called):
        wc_overview(c, indent=indent+"    ")

In [ ]:
wc_overview(wc_node)

# report

In [ ]:
pp.make_report(wc_node)